### Importing python modules

In [10]:
import pandas as pd

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

import datetime 
from dateutil import parser


In [11]:
# functions to get date and year from a string datetime
def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        return None

def to_year(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        year = date.year
        return year
    except:
        return None

# Importing and Adjusting Inflation(CPI) table

    - Create new column in with year
    - get maximum inflation value by year
    - put this in a new dataframe , file

In [12]:
rawInflation = "/Users/deepakgautam/BasicMovieAnalytics/Files/CPIAUCSL.csv"
df_rawInf = pd.read_csv(rawInflation)

In [13]:
df_rawInf["YEAR"] = df_rawInf["DATE"].apply(to_year)

# renaming CPIAUCSL to CPI 
df_rawInf.rename(columns = {'CPIAUCSL' : 'CPI'}, inplace = True)

In [14]:
q = "Select max(CPI) as CPI, YEAR from df_rawInf group by(YEAR)"
df_inf = pysqldf(q)

# Importing Movie Files

In [15]:
rawAllMovies = "/Users/deepakgautam/BasicMovieAnalytics/Files/allMovies.csv"
df_rawAllMovies = pd.read_csv(rawAllMovies, encoding='latin1')

In [17]:
rawIndMovie = "/Users/deepakgautam/BasicMovieAnalytics/Files/indMovieInfo.csv"
df_rawIndMovie = pd.read_csv(rawIndMovie, encoding='latin1')

In [18]:
# combining both movie file into one data frame
q = "Select table1.Rank, table1.MovieName as Movie, table1.Studio, table1.TotalDomestic, \
        table1.TheaterCountTotal, table1.TotalOpeningWeekend, table1.TheaterCountOW, table1.ReleaseDate, \
        table2.Distributor, table2.Genre, table2.RunTime, table2.Budget, table2.MPAA, table2.ForeignSale \
        from df_rawAllMovies as table1 \
        left join df_rawIndMovie as table2 \
         on table1.MovieName = table2.Movie "

rawMainMovie_df = pysqldf(q)

In [19]:
# Getting Release year to join with the inflation table
rawMainMovie_df["ReleaseYear"] = rawMainMovie_df["ReleaseDate"].apply(to_year)


## Joining Movie Table with Adjusted CPI Table
    
    - Make Base Year = 2016 
    - Convert all movies money value info in 2016 terms
    

In [20]:
q = "select table1.*, table2.CPI from rawMainMovie_df as table1 \
     left join df_inf as table2 on table1.ReleaseYear = table2.YEAR"
mainMovie_df = pysqldf(q)


In [21]:
# Creating a inflation multiplier so that we can multiply all money 
# values by this (MAKING BASE YEAR 2016)

q = "select CPI from df_inf where YEAR = 2016 LIMIT 1"
baseYearCPI = pysqldf(q).loc[0][0]
mainMovie_df["InflationMultiplier"] = baseYearCPI/mainMovie_df["CPI"]

## Creating Inflation Adjusted Revenue and Cost Columns

In [22]:
# Creating new Columns with inflation adjusted values
mainMovie_df["AdjTotalDomestic"] = mainMovie_df["TotalDomestic"]*mainMovie_df["InflationMultiplier"]
mainMovie_df["AdjTotalOpeningWeekend"] = mainMovie_df["TotalOpeningWeekend"]*mainMovie_df["InflationMultiplier"]
mainMovie_df["AdjBudget"] = mainMovie_df["Budget"]*mainMovie_df["InflationMultiplier"]
mainMovie_df["AdjForeignSale"] = mainMovie_df["ForeignSale"]*mainMovie_df["InflationMultiplier"]

In [23]:
# Generating Release Month from ReleaseDate
mainMovie_df['ReleaseMonth'] = [parser.parse(i).strftime("%B") for i in mainMovie_df['ReleaseDate']]

In [24]:
# fucntion to get Season Information
def getSeason(month):
    if month in   ['December','January','February']:
        return('Winter')
    elif month in ['March','April','May']:
        return('Spring') 
    elif month in ['June','July','August']:
        return('Summer') 
    elif month in ['September','October','November']:
        return('Fall') 
    else:
        return('None')



In [25]:
# Generating Release Month from ReleaseDate
mainMovie_df['ReleaseSeason'] = [getSeason(i) for i in mainMovie_df['ReleaseMonth']]

In [26]:
#Assigning a Base Genre eg. "Comedy Drama" gets assigned as "Comedy"
mainMovie_df['MainGenre']= [(str(i) if i != None else 'None').partition(' ')[0] for i in mainMovie_df['Genre']]

In [27]:
# Getting Total Sales
mainMovie_df['AdjTotalSales'] = [i for i in ([mainMovie_df['AdjTotalDomestic'].fillna(0) + mainMovie_df['AdjForeignSale'].fillna(0)])[0] ]

In [29]:
mainMovie_df.to_csv('/Users/deepakgautam/BasicMovieAnalytics/Files/mainMovie.csv', encoding='utf-8')